# Building Fast Queries on a CSV

## Introduction

This project is a way to answer a few business questions about an inventory of an online laptop store. 

Our goal is create a class that represent the inventory and do this in an efficient way. The methods in this class will implement the queries that we want to answer about the inventory and we'll preprocess that data to make those queries run faster. Examples of queries are:
* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

It should be noted that the data set is a csv file adapted from [here](https://www.kaggle.com/ionaskel/laptop-prices) by changed IDs and prices made integers.

## Creating the Inventory Class

First, we start implementing a class to represent the inventory. It will take the file name as an argument, read the file, and have attributes as header and rows.

In [3]:
import csv
class Inventory:
    def __init__(self, csv_filename):
        # read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])

#### Code test

In [4]:
inventory = Inventory('laptops.csv')
print('Header:', inventory.header)
print('Number of rows:',len(inventory.rows))

Header: ['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
Number of rows: 1303


## Methods

## Finding a laptop from the Id

This method will take as argument the identifier of the laptop and return the full row of the laptop with that id.

In [5]:
class Inventory:
    def __init__(self, csv_filename):
        # Read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        # take row from id
        for row in self.rows:
            if laptop_id == row[0]:
                return row 
        return

#### Code test

In [6]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id('3362737'))

print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Improving Id Lookups

The previous algorithm requires to look at every single row to find the one that we are looking for. This algorithm has time complexity O(R) where R is the number of rows. 

Using a dictionary, we can have fast lookup properties and it allow us to associate valyes to the keys.

In [9]:
import csv
class Inventory:
    def __init__(self, csv_filename):
        # Read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])
        
        #Implemt dictionary
        self.id_to_row = {}
        for row in self.rows:
            # Fill dictionary
            self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        # take row from id
        for row in self.rows:
            if laptop_id == row[0]:
                return row 
        return
    
    def get_laptop_from_id_fast(self, laptop_id):
        # take row from id faster using dict
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

#### Code test

In [10]:
inventory = Inventory('laptops.csv')
print(inventory.get_laptop_from_id_fast('3362737'))

print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Comparing the Performance

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs.

In [12]:
import time
import random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

inventory = Inventory('laptops.csv')

total_time_no_dict = 0

for id_num in ids:
    start = time.time()
    inventory.get_laptop_from_id(id_num)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0
for id_num in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(id_num)
    end = time.time()
    total_time_dict += end - start
    
print('No dict:', total_time_no_dict, '\nDict:', total_time_dict)

No dict: 0.7302494049072266 
Dict: 0.0040705204010009766


We can see a significant improve in performance.

In [13]:
performance = total_time_no_dict / total_time_dict
print(f'New method is {performance:.0f} time faster.')

New method is 179 time faster.


## Two laptop promotion

In this new method, given a dollar amount, it will check whether it is possible to spend precisely that amount by purchasing up to two laptops. 

This is a way of the store offers a promotion where it gives a gift card and make sure that there is at least one way to spend it in full.

In [15]:
import csv
class Inventory:
    def __init__(self, csv_filename):
        # Read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])
        
        #Implemt dictionary
        self.id_to_row = {}
        for row in self.rows:
            # Fill dictionary
            self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        # Take row from id
        for row in self.rows:
            if laptop_id == row[0]:
                return row 
        return
    
    def get_laptop_from_id_fast(self, laptop_id):
        # take row from id faster using dict
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False

#### Code test

In [16]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars(1000))

print(inventory.check_promotion_dollars(442))

True
False


### Optimizing Laptop Promotion

We can preprocess data to answer the kind of queries that we used in the check_promotion_dollars() to make our code run faster. 

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. 

In [18]:
import csv
class Inventory:
    
    def __init__(self, csv_filename):
        # Read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])
        
        #Implemt dictionary
        self.id_to_row = {}
        for row in self.rows:
            # Fill dictionary
            self.id_to_row[row[0]] = row
        
        # Implement set for prices
        self.prices = set()
        for row in self.rows:
            # Fill set
            self.prices.add(row[-1])
        
                
    def get_laptop_from_id(self, laptop_id):
        # Take row from id
        for row in self.rows:
            if laptop_id == row[0]:
                return row 
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        # Take row from id faster using dict
        if laptop_id in self.id_to_row:
            return self.id_to_row[id_key]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
            
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if dollars - price in self.prices:
                return True
            
        return False

#### Code test

In [19]:
inventory = Inventory('laptops.csv')
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


### Comparing Promotion Functions

In [20]:
prices = [random.randint(100, 5000) for _ in range(100)]

inventory = Inventory('laptops.csv')
total_time_no_set = 0
total_time_set = 0

for value in prices:
    start = time.time()
    inventory.check_promotion_dollars(value)
    end = time.time()
    total_time_no_set += end - start
    
for value in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(value)
    end = time.time()
    total_time_set += end - start
    
print('No set:', total_time_no_set, '\nSet:', total_time_set)

No set: 2.961597442626953 
Set: 0.0013167858123779297


We also can see a significant improve in performance.

In [21]:
performance = total_time_no_set / total_time_set
print(f'New method is {performance:.0f} time faster.')

New method is 2249 time faster.


### Finding Laptops Within a Budget

Now, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

In [37]:
import csv
class Inventory:
    
    def __init__(self, csv_filename):
        # Read file
        with open(csv_filename, 'r', encoding='UTF-8') as file:
            read = list(csv.reader(file))
            
        self.header = read[0]
        self.rows = read[1:]
        
        for row in self.rows:
            # Convert price column to an integer
            row[-1] = int(row[-1])
        
        #Implemt dictionary
        self.id_to_row = {}
        for row in self.rows:
            # Fill dictionary
            self.id_to_row[row[0]] = row
        
        # Implement set for prices
        self.prices = set()
        for row in self.rows:
            # Fill set
            self.prices.add(row[-1])
        
        # Sort by price
        self.rows_by_price = sorted(self.rows, key=lambda x: x[-1])
                
    def get_laptop_from_id(self, laptop_id):
        # take row from id
        for row in self.rows:
            if laptop_id == row[0]:
                return row 
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        # take row from id faster using dict
        if laptop_id in self.id_to_row:
            return self.id_to_row[id_key]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
            
        for row_1 in self.rows:
            for row_2 in self.rows:
                if row_1[-1] + row_2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        
        for price in self.prices:
            if dollars - price in self.prices:
                return True
            
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        # Binary search
        range_start = 0
        range_end = len(self.rows_by_price) - 1
        
        while range_start < range_end:
            range_middle = (range_start +  range_end) // 2
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
                
        if self.rows_by_price[range_start][-1] <= target_price:
            return -1 
        return range_start

#### Code test

In [38]:
inventory = Inventory('laptops.csv')
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


### Conclusion

In this project, we created a class to answer business questions more efficiently by preprocessing the data. Based on the comparison, we saw a huge performance improvement.

However, we decreased time complexity by using more memory. For example, the improved method get_laptop_from_id_fast that stored a dictionary.

------------------------------------